In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression

In [2]:
def selectkbest(indep_X,dep_Y,n):
    test = SelectKBest(score_func=chi2, k=n)
    fit1= test.fit(indep_X,dep_Y)
    selectk_features = fit1.transform(indep_X)
    return selectk_features

In [3]:
def split_scalar(indep_X,dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, y_train, y_test

In [4]:
def r2_prediction(grid,X_test,y_test):
    y_pred = grid.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [5]:
def Linear(X_train,y_train,X_test):       
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return  r2 

In [6]:
def svm(X_train,y_train,X_test):
    param_grid={'kernel':['rbf','poly','sigmond','linear'],'C':[10,100],'gamma':["auto","scale"]}
    grid=GridSearchCV(SVR(),param_grid,refit=True,verbose=3,n_jobs=1)
    grid.fit(X_train, y_train)
    re=grid.cv_results_
    print("The R_score value for best parameter{}:".format(grid.best_params_))
    r2=r2_prediction(grid,X_test,y_test)
    return  r2 

In [7]:
def Decision(X_train,y_train,X_test):
    param_grid={'criterion':['mse','mae','friedman_mse'],'max_features':['auto','sqrt','log2'],'splitter':["best","random"]}
    grid=GridSearchCV(DecisionTreeRegressor(),param_grid,refit=True,verbose=3,n_jobs=1)
    grid.fit(X_train, y_train)
    re=grid.cv_results_
    print("The R_score value for best parameter{}:".format(grid.best_params_))
    r2=r2_prediction(grid,X_test,y_test)
    return  r2

In [8]:
def random(X_train,y_train,X_test):       
    param_grid={'criterion':["squared_error", "absolute_error", 'friedman_mse', 'poisson'],'max_features':['auto','sqrt','log2'],'n_estimators':[10,100]}
    grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=1)
    grid.fit(X_train, y_train)
    re=grid.cv_results_
    print("The R_score value for best parameter{}:".format(grid.best_params_))
    r2=r2_prediction(grid,X_test,y_test)
    return  r2 

In [ ]:
def selectk_regression(acclin, accsvml, accdes, accrf):
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Linear', 'SVMl', 'Decision', 'Random'])

    for number, idex in enumerate(dataframe.index):
        dataframe['Linear'][idex] = acclin[number]
        dataframe['SVMl'][idex] = accsvml[number]
        dataframe['Decision'][idex] = accdes[number]
        dataframe['Random'][idex] = accrf[number]

    return dataframe



acclin = []
accsvml = []
accdes = []
accrf = []

X_train, X_test, y_train, y_test = split_scalar(kbest, dep_Y)
for i in range(kbest.shape[1]):  # iterate over the features
    r2_lin = Linear(X_train[:, i].reshape(-1, 1), y_train, X_test[:, i].reshape(-1, 1))
    acclin.append(r2_lin)

    r2_sl = svm(X_train[:, i].reshape(-1, 1), y_train, X_test[:, i].reshape(-1, 1))
    accsvml.append(r2_sl)

    r2_d = Decision(X_train[:, i].reshape(-1, 1), y_train, X_test[:, i].reshape(-1, 1))
    accdes.append(r2_d)

    r2_r = random(X_train[:, i].reshape(-1, 1), y_train, X_test[:, i].reshape(-1, 1))
    accrf.append(r2_r)

result = selectk_regression(acclin, accsvml, accdes, accrf)

print(result)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.029 total time=   1.3s
[CV 2/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.032 total time=   0.9s
[CV 3/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.034 total time=   0.9s
[CV 4/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.047 total time=   1.1s
[CV 5/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.029 total time=   1.5s
[CV 1/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.029 total time=   0.6s
[CV 2/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.032 total time=   0.3s
[CV 3/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.034 total time=   0.6s
[CV 4/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.047 total time=   0.8s
[CV 5/5] END ....C=10, gamma=auto, kernel=poly;, score=-0.030 total time=   0.3s
[CV 1/5] END ....C=10, gamma=auto, kernel=sigmond;, score=nan total time=   0.0s
[CV 2/5] END ....C=10, gamma=auto, kernel=sigmon

C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\svm\_base.py", line 333, in _dense_f

The R_score value for best parameter{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}:
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END criterion=mse, max_features=auto, splitter=best;, score=-0.931 total time=   0.0s
[CV 2/5] END criterion=mse, max_features=auto, splitter=best;, score=-0.822 total time=   0.0s
[CV 3/5] END criterion=mse, max_features=auto, splitter=best;, score=-0.903 total time=   0.0s
[CV 4/5] END criterion=mse, max_features=auto, splitter=best;, score=-0.675 total time=   0.0s
[CV 5/5] END criterion=mse, max_features=auto, splitter=best;, score=-0.992 total time=   0.0s
[CV 1/5] END criterion=mse, max_features=auto, splitter=random;, score=-0.975 total time=   0.0s
[CV 2/5] END criterion=mse, max_features=auto, splitter=random;, score=-0.975 total time=   0.0s
[CV 3/5] END criterion=mse, max_features=auto, splitter=random;, score=-0.793 total time=   0.0s
[CV 4/5] END criterion=mse, max_features=auto, splitter=random;, score=-0.675 total time=   

C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
C:\Users\Adm

[CV 2/5] END criterion=mse, max_features=sqrt, splitter=best;, score=-0.822 total time=   0.0s
[CV 3/5] END criterion=mse, max_features=sqrt, splitter=best;, score=-0.903 total time=   0.0s
[CV 4/5] END criterion=mse, max_features=sqrt, splitter=best;, score=-0.675 total time=   0.0s
[CV 5/5] END criterion=mse, max_features=sqrt, splitter=best;, score=-0.992 total time=   0.0s
[CV 1/5] END criterion=mse, max_features=sqrt, splitter=random;, score=-0.914 total time=   0.0s
[CV 2/5] END criterion=mse, max_features=sqrt, splitter=random;, score=-0.754 total time=   0.0s
[CV 3/5] END criterion=mse, max_features=sqrt, splitter=random;, score=-0.878 total time=   0.0s
[CV 4/5] END criterion=mse, max_features=sqrt, splitter=random;, score=-0.628 total time=   0.0s
[CV 5/5] END criterion=mse, max_features=sqrt, splitter=random;, score=-1.105 total time=   0.0s
[CV 1/5] END criterion=mse, max_features=log2, splitter=best;, score=-0.931 total time=   0.0s
[CV 2/5] END criterion=mse, max_features

...

C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
C:\Users

[CV 2/5] END criterion=mae, max_features=log2, splitter=random;, score=-0.200 total time=   0.0s
[CV 3/5] END criterion=mae, max_features=log2, splitter=random;, score=-0.204 total time=   0.0s
[CV 4/5] END criterion=mae, max_features=log2, splitter=random;, score=-0.225 total time=   0.0s
[CV 5/5] END criterion=mae, max_features=log2, splitter=random;, score=-0.195 total time=   0.0s
[CV 1/5] END criterion=friedman_mse, max_features=auto, splitter=best;, score=0.044 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_features=auto, splitter=best;, score=0.047 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_features=auto, splitter=best;, score=0.064 total time=   0.0s
[CV 4/5] END criterion=friedman_mse, max_features=auto, splitter=best;, score=0.067 total time=   0.0s
[CV 5/5] END criterion=friedman_mse, max_features=auto, splitter=best;, score=0.113 total time=   0.0s
[CV 1/5] END criterion=friedman_mse, max_features=auto, splitter=random;, score=0.044 total time=

C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\sklearn\tree\_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


[CV 3/5] END criterion=squared_error, max_features=auto, n_estimators=10;, score=0.064 total time=   0.0s
[CV 4/5] END criterion=squared_error, max_features=auto, n_estimators=10;, score=0.067 total time=   0.0s
[CV 5/5] END criterion=squared_error, max_features=auto, n_estimators=10;, score=0.111 total time=   0.0s
[CV 1/5] END criterion=squared_error, max_features=auto, n_estimators=100;, score=0.044 total time=   0.1s
[CV 2/5] END criterion=squared_error, max_features=auto, n_estimators=100;, score=0.047 total time=   0.1s
[CV 3/5] END criterion=squared_error, max_features=auto, n_estimators=100;, score=0.064 total time=   0.1s
[CV 4/5] END criterion=squared_error, max_features=auto, n_estimators=100;, score=0.067 total time=   0.1s
[CV 5/5] END criterion=squared_error, max_features=auto, n_estimators=100;, score=0.113 total time=   0.1s
[CV 1/5] END criterion=squared_error, max_features=sqrt, n_estimators=10;, score=0.044 total time=   0.0s
[CV 2/5] END criterion=squared_error, max

[CV 1/5] END criterion=friedman_mse, max_features=log2, n_estimators=10;, score=0.044 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_features=log2, n_estimators=10;, score=0.047 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_features=log2, n_estimators=10;, score=0.064 total time=   0.0s
[CV 4/5] END criterion=friedman_mse, max_features=log2, n_estimators=10;, score=0.067 total time=   0.0s
[CV 5/5] END criterion=friedman_mse, max_features=log2, n_estimators=10;, score=0.113 total time=   0.0s
[CV 1/5] END criterion=friedman_mse, max_features=log2, n_estimators=100;, score=0.044 total time=   0.1s
[CV 2/5] END criterion=friedman_mse, max_features=log2, n_estimators=100;, score=0.047 total time=   0.1s
[CV 3/5] END criterion=friedman_mse, max_features=log2, n_estimators=100;, score=0.064 total time=   0.1s
[CV 4/5] END criterion=friedman_mse, max_features=log2, n_estimators=100;, score=0.067 total time=   0.1s
[CV 5/5] END criterion=friedman_mse, max_features=l